# Seyed Mohammad Amin Atyabi - 830402014 - HW 7

In [1]:
import json
import pandas
from scipy import io as sio


def split_data(data: pandas.DataFrame):
    splits = []
    for i in range(5):
        splits.append(data.iloc[i:i + 65, :])
    return splits


def train_model(data: pandas.DataFrame):
    model = {}
    P = data.loc[data['DEMOCRAT/LIBERAL'] == 1]
    len_p = len(P) + 2
    N = data.loc[data['DEMOCRAT/LIBERAL'] == 0]
    len_n = len(N) + 2
    for column in data.columns:
        if column == 'DEMOCRAT/LIBERAL': continue
        p_p = P.loc[P[column] == 1]
        p_n = N.loc[N[column] == 0]
        model[column] = {
            '11': (len(p_p) + 1) / len_p,
            '01': 1 - ((len(p_p) + 1) / len_p),
            '10': (len(p_n) + 1) / len_n,
            '00': 1 - ((len(p_n) + 1) / len_n)
        }
    return model


def evaluate_model(model, data: pandas.DataFrame):
    error = 0
    for _, row in data.iterrows():
        predict_1 = 1
        predict_0 = 1
        for column in data.columns:
            if column == 'DEMOCRAT/LIBERAL': continue
            predict_1 *= model[column][f'{row[column]}1']
            predict_0 *= model[column][f'{row[column]}0']
        prediction = 1 if predict_1 > predict_0 else 0
        if prediction != row['DEMOCRAT/LIBERAL']: error += 1
    return error


def print_model(model):
    print(json.dumps(model, indent=4))


columns = [
    'HANDICAPPED-INFANTS',
    'WATER-PROJECT-COST-SHARING',
    'ADOPTION-OF-THE-BUDGET-RESOLUTION',
    'PHYSICIAN-FEE-FREEZE',
    'EL-SALVADOR-AID',
    'RELIGIOUS-GROUPS-IN-SCHOOL',
    'ANTI-SATELLITE-TEST-BAN',
    'AID-TO-NICARAGUAN-CONTRAS',
    'MX-MISSILE',
    'IMMIGRATION',
    'SYNFUELS-CORP-CUTBACK',
    'EDUCATION-SPENDING',
    'SUPERFUND-RIGHT-TO-SUE',
    'CRIME',
    'DUTY-FREE-EXPORTS',
    'EXPORT-ADMINISTRATION-ACT-SOUTH-AFRICA',
    'DEMOCRAT/LIBERAL'
]

data = pandas.DataFrame(sio.loadmat('Data/vote.mat').get('Data'), columns=columns, dtype=int)

splits = split_data(data)

for i in range(len(splits)):
    model = train_model(splits[i])
    avg_error = 0
    for j in range(len(splits)):
        if j == i: continue
        avg_error += evaluate_model(model, splits[j])
    avg_error /= (len(splits) - 1)
    print(avg_error)

34.0
34.0
35.75
35.75
34.75
